In [10]:
import urllib
import pyodbc
import numpy as np
import requests
import logging
import time
import faker
import random
import unicodedata
import pandas as pd
import meteostat
import requests
import time
import sqlalchemy
from apscheduler.schedulers.background import BackgroundScheduler
from datetime import datetime, timedelta
from meteostat import Point, Daily
from sqlalchemy import create_engine, text
from sqlalchemy import create_engine, text, Table, Column, String, Float, Integer, DateTime, MetaData

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)
class DatabaseManager:
    def __init__(self, server, database, username, password):
        """
        Khởi tạo đối tượng DatabaseManager.

        Parameters:
            server (str): Tên server SQL Server.
            database (str): Tên database.
            username (str): Tên tài khoản đăng nhập.
            password (str): Mật khẩu tài khoản.
        """
        self.server = server
        self.database = database
        self.username = username
        self.password = password
        self.engine = None

    def connect_engine(self):
        """
        Tạo kết nối engine cho SQLAlchemy.
        """
        try:
            params = urllib.parse.quote_plus(
                f"DRIVER={{ODBC Driver 17 for SQL Server}};"
                f"SERVER={self.server};"
                f"DATABASE={self.database};"
                f"UID={self.username};"
                f"PWD={self.password};"
                f"Trusted_Connection=no;"
            )
            self.engine = create_engine(f"mssql+pyodbc:///?odbc_connect={params}")
            print("Kết nối SQLAlchemy engine thành công.")
        except Exception as e:
            print(f"Không thể kết nối SQLAlchemy engine. Lỗi: {e}")
    
    def create_database_if_not_exists(self):
        """
        Kiểm tra và tạo database nếu chưa tồn tại.
        """
        conn_str = (
            f"DRIVER={{ODBC Driver 17 for SQL Server}};"
            f"SERVER={self.server};"
            f"UID={self.username};"
            f"PWD={self.password};"
            f"Trusted_Connection=no;"
        )
        try:
            with pyodbc.connect(conn_str, autocommit=True) as conn:
                cursor = conn.cursor()
                cursor.execute(f"IF NOT EXISTS(SELECT * FROM sys.databases WHERE name='{self.database}') CREATE DATABASE {self.database}")
                print(f"Đã kiểm tra và tạo database '{self.database}' nếu chưa tồn tại.")
        except Exception as e:
            print(f"Lỗi khi tạo database: {e}")

    def create_tables(self):
        """
        Tạo các bảng trong database.
        """
        table_queries = {
            "data_sanpham": """
            IF NOT EXISTS (SELECT * FROM sys.objects WHERE object_id = OBJECT_ID(N'data_sanpham') AND type in (N'U'))
            BEGIN
                CREATE TABLE data_sanpham (
                    product_id VARCHAR(20) PRIMARY KEY NOT NULL,
                    brand VARCHAR(100),
                    title VARCHAR(100),
                    type VARCHAR(50),
                    price FLOAT NOT NULL,
                    itemlocation VARCHAR(100) NOT NULL,
                    weather VARCHAR(50) NOT NULL
                );
            END
            """,
            "data_khachhang": """
            IF NOT EXISTS (SELECT * FROM sys.objects WHERE object_id = OBJECT_ID(N'data_khachhang') AND type in (N'U'))
            BEGIN
                CREATE TABLE data_khachhang (
                    customer_id VARCHAR(20) PRIMARY KEY NOT NULL,
                    name VARCHAR(100) NOT NULL,
                    address VARCHAR(100) NOT NULL,
                    job VARCHAR(100) NOT NULL,
                    sex VARCHAR(100) NOT NULL,
                    age BIGINT NOT NULL,
                    phone VARCHAR(100) NOT NULL,
                    email VARCHAR(100) NOT NULL
                );
            END
            """,
            "Orders": """
            IF NOT EXISTS (SELECT * FROM sys.objects WHERE object_id = OBJECT_ID(N'Orders') AND type in (N'U'))
            BEGIN
                CREATE TABLE Orders (
                    order_time DATETIME NOT NULL,
                    order_id VARCHAR(20) PRIMARY KEY NOT NULL,
                    customer_id VARCHAR(20) NOT NULL,
                    Total FLOAT NOT NULL,
                    FOREIGN KEY (customer_id) REFERENCES data_khachhang(customer_id)
                );
            END
            """,
            "Order_detail": """
            IF NOT EXISTS (SELECT * FROM sys.objects WHERE object_id = OBJECT_ID(N'Order_detail') AND type in (N'U'))
            BEGIN
                CREATE TABLE Order_detail (
                    orderdetail_id VARCHAR(20) PRIMARY KEY NOT NULL,
                    order_id VARCHAR(20) NOT NULL,
                    sanpham_id VARCHAR(20) NOT NULL,
                    quanlity BIGINT NOT NULL,
                    price FLOAT NOT NULL,
                    FOREIGN KEY (order_id) REFERENCES Orders(order_id),
                    FOREIGN KEY (sanpham_id) REFERENCES data_sanpham(product_id)
                );
            END
            """
        }

        conn_str = (
            f"DRIVER={{ODBC Driver 17 for SQL Server}};"
            f"SERVER={self.server};"
            f"DATABASE={self.database};"
            f"UID={self.username};"
            f"PWD={self.password};"
            f"Trusted_Connection=no;"
        )

        try:
            with pyodbc.connect(conn_str) as conn:
                cursor = conn.cursor()
                for table_name, query in table_queries.items():
                    cursor.execute(query)
                    print(f"Đã kiểm tra và tạo bảng '{table_name}' nếu chưa tồn tại.")
                conn.commit()
        except Exception as e:
            print(f"Lỗi khi tạo bảng: {e}")
if __name__ == "__main__":
    server = r'DESKTOP-Q6B5CSD\NAMNH' # Thay thế lại server của bạn
    database = 'Capstone2' #Tên Database của bạn
    username = 'sa' #Tài khoản của bạn
    password = 'Nam@15092003' #Mật khẩu của bạn

    db_manager = DatabaseManager(server, database, username, password)
    db_manager.create_database_if_not_exists()
    db_manager.create_tables()
    db_manager.connect_engine()
def my_long_running_task():
    def Extract_process_product_data(file_path):
        sp1 = pd.read_csv(file_path)
        sp1['brand'] = sp1['brand'].str.title()
        sp1 = sp1.drop(columns=['priceWithCurrency', 'available', 'availableText', 'sold', 'lastUpdated']) # Remove unnecessary columns
        sp1['product_id'] = sp1['product_id'].astype(str).str.slice(0, 20)
        sp1['itemLocation'] = sp1['itemLocation'].str.split(', ').str[-1] # Split and extract the country from the 'itemLocation' column
        # Chuyển cột title, brand, và type thành chữ thường
        sp1['title'] = sp1['title'].astype(str).str.lower()
        sp1['brand'] = sp1['brand'].astype(str).str.lower()
        sp1['type'] = sp1['type'].fillna('không xác định').astype(str).str.lower()

        # Lấy các giá trị duy nhất của cột brand và type
        unique_brands = sp1['brand'].unique()
        unique_types = sp1['type'].unique()

        # Khởi tạo danh sách để lưu lại các giá trị bị thay thế và vị trí của chúng
        replaced_values_brand = []
        replaced_values_type = []

        # Hàm để thay thế giá trị nếu tìm thấy trong title
        def replace_value_in_column(df, index, current_value, column_name, unique_values, replaced_values):
            title_value = df.at[index, 'title']

            # Kiểm tra nếu giá trị hiện tại nằm trong title
            if current_value in title_value:
                return current_value  # Nếu có, giữ nguyên giá trị

            # Nếu không tìm thấy, kiểm tra qua các giá trị duy nhất trong cột tương ứng
            for value in unique_values:
                if value in title_value:
                    # Lưu lại giá trị bị thay thế và vị trí của nó
                    replaced_values.append({'index': index, f'old_{column_name}': current_value, f'new_{column_name}': value})
                    return value  # Trả về giá trị mới khi đã thay thế

            return current_value  # Giữ nguyên nếu không thay thế

        # Duyệt qua từng hàng trong DataFrame và xử lý cả hai cột brand và type
        sp1['type'] = sp1.apply(lambda row: replace_value_in_column(sp1, row.name, row['type'], 'type', unique_types, replaced_values_type), axis=1)
        # Thay thế cột 'brand' bằng apply
        sp1['brand'] = sp1.apply(lambda row: replace_value_in_column(sp1, row.name, row['brand'], 'brand', unique_brands, replaced_values_brand), axis=1)

        # Hàm để kiểm tra và thay thế lại giá trị cũ nếu cần thiết
        def restore_old_value(df, replaced_values, column_name):
            for item in replaced_values:
                index = item['index']
                old_value = item[f'old_{column_name}']

                # Kiểm tra xem giá trị cũ có xuất hiện trong bất kỳ title nào không
                found_in_title = any(old_value in title for title in df['title'])

                # Nếu có xuất hiện trong title, thay thế giá trị hiện tại bằng giá trị cũ
                if found_in_title:
                    df.at[index, column_name] = old_value

        # Kiểm tra và phục hồi giá trị cũ cho cả brand và type nếu cần
        restore_old_value(sp1, replaced_values_brand, 'brand')
        restore_old_value(sp1, replaced_values_type, 'type')
        sp1['type'] = sp1['type'].astype(str).str.lower().str.strip()  # Chuyển cột type thành chữ thường và xóa khoảng trắng

    # Từ điển các giá trị sai
        incorrect_types = {
            'eau de parfum': 'eau de parfum',
            'eau de perfum': 'eau de parfum',
            'eau de parfume': 'eau de parfum',
            'eau de perfume': 'eau de parfum',
            'edp': 'eau de parfum',
            'eau parfum': 'eau de parfum',
            'eau parfume': 'eau de parfum',
            'au parfume': 'eau de parfum',
            'au perfume': 'eau de parfum',
            'de parfume': 'eau de parfum',
            'de perfume': 'eau de parfum',
            'de perfuma': 'eau de parfum',
            'eau de toillete': 'eau de toilette',
            'eau de toilette': 'eau de toilette',
            'eau de tollette': 'eau de toilette',
            'eau toilette': 'eau de toilette',
            'de toilette': 'eau de toilette',
            'de tollette': 'eau de toilette',
            'edt': 'eau de toilette',
            'au toilette': 'eau de toilette',
            'au tollette': 'eau de toilette',
            'tollette': 'eau de toilette',
            'eau de cologne': 'eau de cologne',
            'eau de cologne': 'eau de cologne',
            'edc': 'eau de cologne',
            'cologne': 'eau de cologne',
            'eau cologne': 'eau de cologne',
            'de cologne': 'eau de cologne',
            'au cologne': 'eau de cologne',
            'de cologna': 'eau de cologne',
            'au cologna': 'eau de cologne',
            'perfume': 'perfume',
            'perfuma': 'perfume',
            'perfumar': 'perfume',
            'perfumer': 'perfume',
            'parfum': 'perfume',

        }

        # Duyệt qua từng hàng trong DataFrame
        for index in range(len(sp1)):
            current_type = sp1.at[index, 'type']  # Lấy giá trị của cột type ở hàng hiện tại

            # In ra giá trị ban đầu
            None #print(f"Giá trị ban đầu: '{current_type}'", end=" -> ")

            # Kiểm tra xem current_type có trong từ điển incorrect_types không
            found_correction = False  # Biến để theo dõi xem có tìm thấy giá trị sửa đổi không
            for incorrect, correct in incorrect_types.items():
                if incorrect in current_type:  # Kiểm tra xem giá trị sai có trong chuỗi không
                    None
                    sp1.at[index, 'type'] = correct  # Cập nhật giá trị trong DataFrame
                    found_correction = True  # Đánh dấu là đã tìm thấy sửa đổi
                    break  # Ngừng kiểm tra sau khi tìm thấy sửa đổi

            if not found_correction:  # Nếu không tìm thấy
                if current_type not in ["eau de parfum", "eau de toilette", "eau de cologne"]:
                    None
                    sp1.at[index, 'type'] = "eau de cologne"  # Chuyển về eau de cologne
                else:
                    None
            continue
        weather_mapping = {
            'perfume': 'Cold',
            'eau de parfum': 'Cool',
            'eau de toilette': 'Mild & Comfortable',
            'eau de cologne': 'Hot'}
        sp1['weather'] = sp1['type'].map(weather_mapping).fillna('Không xác định')
        #sp1['itemLocation'] = sp1['itemLocation'].str.split(', ').str[-1]
        sp1['brand'] = sp1['brand'].str.title()
        sp1['title'] = sp1['title'].str.title()
        sp1['type'] = sp1['type'].str.title()
        return sp1
    def get_max_id(table_name: str, id_column: str, prefix_length: int):
        """
        Hàm lấy giá trị ID lớn nhất từ bảng dữ liệu.

        Parameters:
        - table_name: Tên bảng (e.g., 'data_khachhang', 'Orders', 'Order_detail').
        - id_column: Tên cột chứa ID (e.g., 'customer_id', 'order_id', 'orderdetail_id').
        - prefix_length: Độ dài của tiền tố trong ID (e.g., 1 cho customer_id, 2 cho order_id).

        Returns:
        - int: Giá trị ID lớn nhất trong bảng.
        """
        query = text(f"SELECT MAX(CONVERT(int, SUBSTRING({id_column}, {prefix_length + 1}, LEN({id_column})))) AS max_id FROM {table_name}")
        with db_manager.engine.connect() as connection:
            result = connection.execute(query)
            max_id = result.scalar() or 0  # Trả về 0 nếu không có ID nào
        return max_id
    def generate_unique_id(existing_ids, existing_max_id, i, data_id, prefix):
        """
        Hàm chung để tạo ID duy nhất.

        Args:
        - existing_ids (list): Danh sách các ID hiện có trong cơ sở dữ liệu.
        - existing_max_id (int): ID lớn nhất hiện tại.
        - i (int): Tham số xác định trạng thái (1: dùng existing_ids, khác 1: dùng data_id).
        - data_id (list): Danh sách các ID mới để kiểm tra.
        - prefix (str): Tiền tố của ID ('C' hoặc 'O').

        Returns:
        - str: ID mới với định dạng PREFIX000000X.
        """
        # Xác định danh sách ID hiện có
        if i == 1:
            existing_ids_set = set(existing_ids)
            new_id = existing_max_id + 1
        else:
            existing_ids_set = set(data_id)
            new_id = existing_max_id + 1

        while True:
            # Tạo ID với tiền tố và định dạng cố định
            unique_id = f"{prefix}{new_id:010d}"
            if unique_id not in existing_ids_set:
                return unique_id
            new_id += 1

    def Create_customer(existing_ids, engine):
        fake = faker.Faker('vi_VN')

        # Các đầu số điện thoại hợp lệ ở Việt Nam
        phone_prefixes = [
        "039", "038", "037", "036", "035", "034", "033", "032",
        "083", "084", "085", "081", "088", "082", "070", "079",
        "077", "076", "078", "052", "056", "058"
        ]

        # Danh sách tên phổ biến cho nam và nữ
        male_names = [
            "Anh", "Bảo", "Dũng", "Đức", "Hưng", "Hải", "Khoa", "Khánh", "Minh",
            "Nguyên", "Nhân", "Phong", "Phúc", "Quân", "Sơn", "Thắng", "Thiện", "Trường", "Tuấn", "Vinh"
        ]

        female_names = [
            "Anh", "Bích", "Chi", "Dung", "Duyên", "Hạnh", "Hoa", "Hương", "Lan",
            "Linh", "Mai", "Ngân", "Ngọc", "Nhi", "Phương", "Quỳnh", "Thu", "Trang", "Trúc", "Vy"
        ]

        # Danh sách tên đường, quận/huyện và thành phố
        street_names = [
            "Nguyen Hue", "Le Loi", "Tran Hung Dao", "Nguyen Trai", "Cach Mang Thang 8",
            "Hai Ba Trung", "Pham Ngu Lao", "Ly Thuong Kiet", "Dien Bien Phu", "Hoang Dieu",
            "Vo Thi Sau", "Phan Xich Long", "Ton Duc Thang", "Pham Van Dong", "Nguyen Van Cu",
            "Truong Chinh", "Quang Trung", "Hoang Sa", "Truong Sa", "Ly Tu Trong", "Hung Vuong"
        ]

        districts = {
            "Ho Chi Minh": [
                "Quan 1", "Quan 3", "Quan 5", "Quan 7", "Quan Tan Binh", "Quan Binh Thanh", "Quan Go Vap",
                "Quan Phu Nhuan", "Quan Thu Duc", "Huyen Binh Chanh", "Quan 2", "Quan 4", "Quan 9", "Quan 10"
            ],
            "Ha Noi": [
                "Quan Hoan Kiem", "Quan Dong Da", "Quan Ba Dinh", "Quan Hai Ba Trung", "Quan Cau Giay",
                "Quan Tay Ho", "Quan Long Bien", "Quan Ha Dong", "Quan Thanh Xuan", "Huyen Gia Lam", "Huyen Dong Anh"
            ],
            "Da Nang": [
                "Quan Hai Chau", "Quan Son Tra", "Quan Ngu Hanh Son", "Quan Cam Le", "Quan Thanh Khe", "Quan Lien Chieu"
            ],
            "Hai Phong": [
                "Quan Le Chan", "Quan Ngo Quyen", "Quan Hong Bang", "Quan Kien An", "Quan Do Son", "Quan Duong Kinh"
            ],
            "Ca Mau": [
                "Huyen Cai Nuoc", "Huyen Dam Doi", "Huyen Nam Can", "Huyen Ngoc Hien", "Huyen Phu Tan","Huyen Thoi Binh","Huyen Tran Van Thoi","Huyen U Minh"
            ],
            "Nha Trang": [
                "Quan Vinh Hai", "Quan Vinh Hoa", "Quan Phuoc Long", "Quan Phuoc Hai", "Quan Ngoc Hiep"
            ],
            "Hue": [
                "Quan Thuan An", "Quan Phu Vang", "Quan Huong Tra", "Quan Huong Thuy", "Quan Phu Nhuan"
            ]
        }

        # Hàm sinh số điện thoại Việt Nam
        def generate_vietnam_phone():
            prefix = random.choice(phone_prefixes)  # Chọn ngẫu nhiên một đầu số
            suffix = ''.join([str(random.randint(0, 9)) for _ in range(7)])  # Sinh chính xác 7 số ngẫu nhiên
            return f"{prefix}{suffix}"

        # Hàm tạo địa chỉ ngẫu nhiên
        def generate_vietnam_address():
            street_number = random.randint(1, 400)  # Số nhà từ 1 đến 400
            street_name = random.choice(street_names)
            city = random.choice(list(districts.keys()))
            district = random.choice(districts[city])
            return f"{street_number} {street_name}, {district}, {city}"

        # Hàm loại bỏ dấu tiếng Việt
        def remove_vietnamese_accents(text):
            nfkd_form = unicodedata.normalize('NFKD', text)
            return ''.join([c for c in nfkd_form if not unicodedata.combining(c)])
        # Hàm tạo email
        def generate_email(full_name):
            name_parts = full_name.split()  # Tách họ và tên
            first_name = remove_vietnamese_accents(name_parts[-1])  # Tên không dấu
            last_name = remove_vietnamese_accents(name_parts[0])    # Họ không dấu

            # Thêm số ngẫu nhiên từ 2 đến 4 chữ số và một số bổ sung để tránh lặp lại
            random_number = ''.join([str(random.randint(0, 9)) for _ in range(random.randint(2, 4))])

            # Chọn ngẫu nhiên một cấu trúc email
            email_structure = random.choice([
                f"{first_name}{random_number}@gmail.com",
                f"{last_name}{random_number}@yahoo.com",
                f"{first_name}.{last_name}{random_number}@outlook.com",
                f"{first_name[0]}{last_name}{random_number}@mail.com",
                f"{last_name}.{first_name}{random_number}@gmail.com"
            ])

            return email_structure.lower()

        # Hàm sinh tên đầy đủ
        def generate_full_name_by_gender(gender):
            if gender == 'Nam':
                last_name = remove_vietnamese_accents(fake.last_name_male())  # Họ cho nam (không dấu)
                first_name = random.choice(male_names)  # Tên cho nam
            else:
                last_name = remove_vietnamese_accents(fake.last_name_female())  # Họ cho nữ (không dấu)
                first_name = random.choice(female_names)  # Tên cho nữ

            full_name = f"{last_name} {first_name}"
            return full_name

        # Hàm lấy customer_id lớn nhất trong cơ sở dữ liệu
        max_customer_id = get_max_id('data_khachhang', 'customer_id', 1)
        customer_data = []
        data_id=[]

        # Sinh 100 khách hàng giả
        for i in range(1,100):
            # Kiểm tra và tạo mã khách hàng duy nhất
            gender = fake.random_element(elements=('Nam', 'Nữ'))
            full_name = generate_full_name_by_gender(gender)
            customer_id = generate_unique_id(existing_ids, max_customer_id,i,data_id,prefix="C")
            data_id.append(customer_id)
            customer = {
                "customer_id": customer_id,
                "name": remove_vietnamese_accents(full_name),  # Tên không dấu
                "address": remove_vietnamese_accents(generate_vietnam_address()),  # Địa chỉ không dấu
                "job": remove_vietnamese_accents(fake.job()),  # Job không dấu
                "sex": "Male" if gender == "Nam" else "Female",  # Chuyển giới tính
                "age": fake.random_int(min=18, max=65),
                "phone": generate_vietnam_phone(),
                "email": generate_email(full_name)
            }
            customer_data.append(customer)

        # Tạo DataFrame từ dữ liệu giả
        df_customers = pd.DataFrame(customer_data)
        return df_customers
    def create_order_data(sp1, df_customers, existing_ids, engine):
        def generate_order_datetime(weather_label):
            chosen_months = {
                "Cold": [11, 12, 1],
                "Cool": [2, 9, 10],
                "Mild & Comfortable": [3, 4],
                "Hot": [5, 6, 7, 8]
            }.get(weather_label, [1, 2, 3])  # Default to some months if label is missing

            chosen_month = random.choice(chosen_months)
            random_day = random.randint(1, 31 if chosen_month in [1, 3, 5, 7, 8, 10, 12] else (28 if chosen_month == 2 else 30))
            order_date = datetime(2023, chosen_month, random_day)
            random_time = timedelta(
                hours=random.randint(0, 23),
                minutes=random.randint(0, 59),
                seconds=random.randint(0, 59)
            )
            return order_date + random_time
        def generate_random_date():
            month_probs = {
              1: 0.1,
              2: 0.07,
              3: 0.07,
              4: 0.05,
              5: 0.02,
              6: 0.02,
              7: 0.02,
              8: 0.02,
              9: 0.1,
              10: 0.16,
              11: 0.15,
              12: 0.2
          }

            months = list(month_probs.keys())
            probs = list(month_probs.values())

            if not np.isclose(sum(probs), 1.0):
                  probs = [p / sum(probs) for p in probs]

            chosen_month = np.random.choice(months, p=probs)

            if chosen_month in [1, 3, 5, 7, 8, 10, 12]:
                random_day = random.randint(1, 31)
            elif chosen_month == 2:
                random_day = random.randint(1, 28)
            else:
                random_day = random.randint(1, 30)

            return datetime(2023, chosen_month, random_day)

        # Retrieve maximum existing order_id
        max_order_id = get_max_id('Orders', 'order_id', 1)
        order_list = []
        data_id = []

        # Generate 100 orders
        for i in range(1, 100):
            order_id = generate_unique_id(existing_ids,  max_order_id,i, data_id,prefix="O")
            data_id.append(order_id)

            # Chọn customer_id ngẫu nhiên từ df_customers
            customer_id = random.choice(df_customers['customer_id'])

            # Thiết lập ngày đặt hàng mặc định
            order_date = generate_random_date()  # Bạn có thể thay đổi logic này nếu cần

            order_list.append({
                'order_time': order_date,
                'order_id': order_id,
                'customer_id': customer_id,
                'Total': 0  # Giá trị tổng sẽ được tính ở bước sau
            })

        return pd.DataFrame(order_list)

    def create_order_details(sp1, data_orders, existing_ids, engine):
        max_orderdetail_id = get_max_id('Order_detail', 'orderdetail_id', 2)
        # Tạo ID mới cho đơn hàng
        order_detail_list = []
        def generate_order_id(i):
            return f"D{str(i).zfill(10)}"

        # Biến đếm cho ID chi tiết đơn
        detail_id_counter = 1

        # Số lượng đơn hàng bạn đã tạo
        num_orders = 100  # Ví dụ, số lượng đơn hàng đã tạo từ trước

        # Xác suất cho số lượng sản phẩm
        num_products = [1, 2, 3, 4, 5, 6, 7]
        probabilities = [0.2, 0.15, 0.15, 0.13, 0.11, 0.08, 0.07]
        data_orderdetail_id=[]
        dem=1
        # Tạo dữ liệu chi tiết đơn hàng
        for order_index in range(1, num_orders + 1):
            # Chọn order_id ngẫu nhiên từ data_orders
            order_id = random.choice(data_orders['order_id'])
            # Lấy số lượng sản phẩm theo xác suất
            num_products_in_order = random.choices(num_products, weights=probabilities, k=1)[0]

            # Lưu trữ các ID sản phẩm đã chọn và nhãn tương ứng
            chosen_products = []

            for _ in range(num_products_in_order):
                # Chọn ID sản phẩm ngẫu nhiên
                product_id = random.choice(sp1['product_id'])
                chosen_products.append(product_id)

            # Lấy nhãn thời tiết của các sản phẩm đã chọn
            labels = sp1.loc[sp1['product_id'].isin(chosen_products), 'weather'].values

            # Đếm tần suất nhãn
            label_counts = pd.Series(labels).value_counts()

            # Lấy nhãn đa số
            predominant_label = label_counts.idxmax()

            # Thêm sản phẩm theo quy tắc đã định nghĩa
            for product_id in chosen_products:
                # Lấy giá của sản phẩm tương ứng
                product_price = sp1.loc[sp1['product_id'] == product_id, 'price'].values[0]
                # Tạo số lượng sản phẩm ngẫu nhiên
                quantity = random.randint(1, 5)  # Hoặc theo logic của bạn
                new_order_id = generate_unique_id(existing_ids, max_orderdetail_id, dem, data_orderdetail_id, 'OD')
                data_orderdetail_id.append(new_order_id)
                # Thêm dữ liệu chi tiết đơn hàng vào danh sách
                order_detail_list.append({
                    'orderdetail_id': new_order_id,#str(uuid.uuid4()),
                    'order_id': order_id,
                    'sanpham_id': product_id,
                    'quanlity': quantity,
                    'price': product_price
                })

                # Tăng biến đếm ID chi tiết đơn
                dem+=1
                detail_id_counter += 1

            # Áp dụng quy tắc phân phối cho ID sản phẩm theo nhãn đa số
            if len(chosen_products) > 1:  # Chỉ áp dụng nếu có nhiều sản phẩm
                dem+=1
                new_order_id = generate_unique_id(existing_ids, max_orderdetail_id, dem, data_orderdetail_id, 'OD')
                data_orderdetail_id.append(new_order_id)
                if predominant_label == "Cold":
                    # Thêm 1 sản phẩm có nhãn Ấm áp, dễ chịu (40%) hoặc Se lạnh (60%)
                    if random.random() < 0.4:
                        additional_product = random.choice(sp1[sp1['weather'] == "Mild & Comfortable"]['product_id'].tolist())
                    else:
                        additional_product = random.choice(sp1[sp1['weather'] == "Cool"]['product_id'].tolist())
                    order_detail_list.append({
                        'orderdetail_id': new_order_id,#str(uuid.uuid4()),
                        'order_id': order_id,
                        'sanpham_id': additional_product,
                        'quanlity': random.randint(1, 5),
                        'price': sp1.loc[sp1['product_id'] == additional_product, 'price'].values[0]
                    })
                    detail_id_counter += 1

                elif predominant_label == "Cool":
                    additional_product = random.choice(sp1[sp1['weather'] == "Cold"]['product_id'].tolist())
                    order_detail_list.append({
                        'orderdetail_id': new_order_id,#str(uuid.uuid4()),
                        'order_id': order_id,
                        'sanpham_id': additional_product,
                        'quanlity': random.randint(1, 5),
                        'price': sp1.loc[sp1['product_id'] == additional_product, 'price'].values[0]
                    })
                    detail_id_counter += 1

                elif predominant_label == "Mild & Comfortable":
                    additional_product = random.choice(sp1[sp1['weather'] == "Hot"]['product_id'].tolist())
                    order_detail_list.append({
                        'orderdetail_id': new_order_id,#str(uuid.uuid4()),
                        'order_id': order_id,
                        'sanpham_id': additional_product,
                        'quanlity': random.randint(1, 5),
                        'price': sp1.loc[sp1['product_id'] == additional_product, 'price'].values[0]
                    })
                    detail_id_counter += 1

                elif predominant_label == "Hot":
                    additional_product = random.choice(sp1[sp1['weather'] == "Cool"]['product_id'].tolist())
                    order_detail_list.append({
                        'orderdetail_id': new_order_id,#str(uuid.uuid4()),
                        'order_id': order_id,
                        'sanpham_id': additional_product,
                        'quanlity': random.randint(1, 5),
                        'price': sp1.loc[sp1['product_id'] == additional_product, 'price'].values[0]
                    })
                    detail_id_counter += 1
        return pd.DataFrame(order_detail_list)

    def Getlocation_weather(df_customers):
        coordinates = {}

        def get_location():
            # Lấy danh sách địa chỉ và tách ra phần thành phố
            new_list = []
            for address in df_customers['address']:
                last_part = address.rsplit(",", 1)[-1].strip()
                new_list.append(last_part)

            unique_list = list(set(new_list))  # Lấy danh sách thành phố duy nhất

            headers = {
                'User-Agent': 'Capstone2/1.0 (nam.studyjob@gmail.com)'  # Đặt tên ứng dụng và email của bạn
            }

            def get_lat_lon(city):
                url = f'https://nominatim.openstreetmap.org/search?city={city}&format=json'
                try:
                    response = requests.get(url, headers=headers)
                    response.raise_for_status()  # Kiểm tra lỗi HTTP
                    data = response.json()
                    if data:
                        return float(data[0]['lat']), float(data[0]['lon'])
                except requests.exceptions.RequestException as e:
                        print(f"Lỗi khi gửi yêu cầu cho {city}: {e}")
                return None, None

            # Lưu tọa độ vào từ điển và danh sách để tạo DataFrame
            coordinate_data = []
            for city in unique_list:
                lat, lon = get_lat_lon(city)
                if lat is not None and lon is not None:  # Kiểm tra nếu lat và lon hợp lệ
                    coordinates[city] = (lat, lon)
                    coordinate_data.append({'city': city, 'lat': lat, 'lon': lon})
                    print(f"Fetched coordinates for {city}: ({lat}, {lon})")
                else:
                    print(f"Could not fetch coordinates for {city}")
                time.sleep(1.5)  # Thêm delay giữa các request

            # Tạo DataFrame Coordinates
            df_coordinates = pd.DataFrame(coordinate_data)
            return df_coordinates

        def get_weather(coordinates):
            city_data = {}
            end_date = datetime.now()
            start_date = end_date - timedelta(days=3*365)  # 3 năm trước

            for city, (lat, lon) in coordinates.items():
                point = Point(lat, lon)
                weather_data = Daily(point, start=start_date, end=end_date)
                weather_data = weather_data.fetch()

                # Thay thế giá trị null bằng 0
                weather_data.fillna(0, inplace=True)

                # Gán dữ liệu thời tiết vào từ điển
                city_data[city] = weather_data
                print(f"Weather data for {city} has been assigned.")

            return city_data

        # Lấy tọa độ các thành phố
        df_coordinates = get_location()

        # Lấy dữ liệu thời tiết dựa trên tọa độ các thành phố
        city_weather_data = get_weather(coordinates)

        return df_coordinates, city_weather_data

    # 1. Load customer data into the database
    existing_customer_ids = pd.read_sql('SELECT customer_id FROM data_khachhang', con=db_manager.engine)['customer_id'].tolist()
    df_customers = Create_customer(existing_customer_ids, db_manager.engine)  # Thêm tham số engine ở đây
    df_customers.to_sql('data_khachhang', con=db_manager.engine, if_exists='append', index=False)
    print(df_customers)

    # 2. Load sản phẩm từ file
    file_path = 'data_sanpham.csv'
    sp1 = Extract_process_product_data(file_path)  # Giả sử bạn đã có hàm này để tạo data sản phẩm
    # 2. Kiểm tra và chèn sản phẩm vào cơ sở dữ liệu
    existing_ids = pd.read_sql('SELECT product_id FROM data_sanpham', con=db_manager.engine)
    existing_ids_set = set(existing_ids['product_id'])  # Tạo tập hợp ID đã tồn tại
    # 3. Kiểm tra từng sản phẩm và chèn vào cơ sở dữ liệu
    new_products_count = 0  # Biến đếm số sản phẩm mới được thêm vào
    for index, row in sp1.iterrows():
        product_id = row['product_id']  # Lấy ID sản phẩm từ dòng hiện tại

        if product_id not in existing_ids_set:
            # Chèn sản phẩm mới vào cơ sở dữ liệu
            row.to_frame().T.to_sql('data_sanpham', con=db_manager.engine, if_exists='append', index=False)
            new_products_count += 1
            print(f"Chèn sản phẩm mới với ID: {product_id}")
        else:
            # Nếu ID đã tồn tại, bỏ qua
            print(f"Sản phẩm với ID: {product_id} đã tồn tại. Bỏ qua.")
    # In ra số lượng sản phẩm mới đã thêm
    print(f"Đã thêm {new_products_count} sản phẩm mới vào cơ sở dữ liệu.")

    # 3. Load order data into the database
    existing_order_ids = pd.read_sql('SELECT order_id FROM Orders', con=db_manager.engine)['order_id'].tolist() 
    data_orders = create_order_data(sp1, df_customers,existing_order_ids, db_manager.engine)  # Thêm tham số engine ở đây 
    data_orders.to_sql('Orders', con=db_manager.engine, if_exists='append', index=False) 
    print(data_orders)

    # 4. Load order details data into the database
    existing_ordedetail_ids = pd.read_sql('SELECT orderdetail_id FROM  Order_detail', con=db_manager.engine)['orderdetail_id'].tolist() 
    data_orderdetail = create_order_details(sp1, data_orders, existing_ordedetail_ids, db_manager.engine)  # Thêm tham số engine ở đây 
    data_orderdetail.to_sql('Order_detail', con=db_manager.engine, if_exists='append', index=False) 
    print(data_orderdetail)

    # Câu lệnh SQL để cập nhật cột Total trong bảng data_orders
    update_query = text("""
        WITH CTE AS (
            SELECT 
                o.order_id,
                COALESCE(d.price, (SELECT AVG(price) FROM  Order_detail)) AS price_filled,
                COALESCE(d.quanlity, (SELECT AVG(quanlity) FROM  Order_detail)) AS quanlity_filled
            FROM 
                Orders o
            LEFT JOIN 
                Order_detail d ON o.order_id = d.order_id
        )
        UPDATE Orders
        SET Total = cte.price_filled * cte.quanlity_filled
        FROM CTE cte
        WHERE Orders.order_id = cte.order_id
    """)

    # Thực hiện câu lệnh cập nhật
    with db_manager.engine.connect() as connection:
        # Thực hiện câu lệnh
        connection.execute(update_query)
        # Commit thay đổi
        connection.commit()

    # 5. Get coordinates and weather data
    df_coordinates, city_weather_data = Getlocation_weather(df_customers)

    # 6. Load coordinates data into the database
    df_coordinates.to_sql('data_toado', con=db_manager.engine, if_exists='append', index=False)
    def create_weather_table(table_name, engine):
        metadata = MetaData()
        table = Table(
            table_name, metadata,
            Column('time', DateTime, nullable=False, primary_key=True),  # Keep 'time' as primary key
            Column('tavg', Float),
            Column('tmin', Float),
            Column('tmax', Float),
            Column('prcp', Float),
            Column('snow', Float),
            Column('wdir', Float),
            Column('wspd', Float),
            Column('wpgt', Float),
            Column('pres', Float),
            Column('tsun', Float),
        )
        metadata.create_all(engine)  # Create the table if it doesn't exist

    # Create weather tables for each city
    city_data_tables = [
        ('Ho Chi Minh', 'data_[Ho Chi Minh]'),
        ('Ha Noi', 'data_[Ha Noi]'),
        ('Da Nang', 'data_[Da Nang]'),
        ('Ca Mau', 'data_[Ca Mau]'),
        ('Nha Trang', 'data_[Nha Trang]'),
        ('Hai Phong', 'data_[Hai Phong]'),
        ('Hue', 'data_[Hue]')
    ]

    for _, table_name in city_data_tables:
        create_weather_table(table_name, db_manager.engine)

    # Function to retrieve existing data
    def get_existing_data(table_name):
        try:
            return pd.read_sql(f'SELECT * FROM [dbo].[{table_name}]]', con=db_manager.engine)
        except Exception as e:
            print(f"Error retrieving data from {table_name}: {e}")
            return pd.DataFrame()

    # Example data fetching from the city_weather_data dictionary
    hcm_data = city_weather_data.get("Ho Chi Minh")
    hanoi_data = city_weather_data.get("Ha Noi")
    danang_data = city_weather_data.get("Da Nang")
    camau_data = city_weather_data.get("Ca Mau")
    nhatrang_data = city_weather_data.get("Nha Trang")
    haiphong_data = city_weather_data.get("Hai Phong")
    hue_data = city_weather_data.get("Hue")

    # Retrieve existing data for each city
    existing_hcm_data = get_existing_data('data_[Ho Chi Minh]')
    existing_hanoi_data = get_existing_data('data_[Ha Noi]')
    existing_danang_data = get_existing_data('data_[Da Nang]')
    existing_camau_data = get_existing_data('data_[Ca Mau]')
    existing_nhatrang_data = get_existing_data('data_[Nha Trang]')
    existing_haiphong_data = get_existing_data('data_[Hai Phong]')
    existing_hue_data = get_existing_data('data_[Hue]')

    # Define function to save weather data without duplicates
    def save_weather_data(city_data, table_name, existing_data):
        if city_data is not None:
            # Use .index to reference the 'time' values, ensure 'time' is in the existing data
            new_data = city_data[~city_data.index.isin(existing_data['time'])]
            if not new_data.empty:
                new_data.reset_index(inplace=True)  # Reset index if you want to save 'time' as a column
                new_data.to_sql(table_name, con=db_manager.engine, if_exists='append', index=False)

    # Save data for each city
    save_weather_data(hcm_data, 'data_[Ho Chi Minh]', existing_hcm_data)
    save_weather_data(hanoi_data, 'data_[Ha Noi]', existing_hanoi_data)
    save_weather_data(danang_data, 'data_[Da Nang]', existing_danang_data)
    save_weather_data(camau_data, 'data_[Ca Mau]', existing_camau_data)
    save_weather_data(nhatrang_data, 'data_[Nha Trang]', existing_nhatrang_data)
    save_weather_data(haiphong_data, 'data_[Hai Phong]', existing_haiphong_data)
    save_weather_data(hue_data, 'data_[Hue]', existing_hue_data)
scheduler = BackgroundScheduler()
scheduler.add_job(my_long_running_task, 'interval', seconds=300)  # Chạy mỗi 300 giây
scheduler.start()

# Giữ cho ứng dụng chạy
try:
    while True:
        time.sleep(1)
except (KeyboardInterrupt, SystemExit):
    scheduler.shutdown()
    logger.info("Scheduler đã dừng lại!")


Đã kiểm tra và tạo database 'Capstone2' nếu chưa tồn tại.
Đã kiểm tra và tạo bảng 'data_sanpham' nếu chưa tồn tại.
Đã kiểm tra và tạo bảng 'data_khachhang' nếu chưa tồn tại.
Đã kiểm tra và tạo bảng 'Orders' nếu chưa tồn tại.
Đã kiểm tra và tạo bảng 'Order_detail' nếu chưa tồn tại.
Kết nối SQLAlchemy engine thành công.
    customer_id         name                                       address  \
0   C0000000694    Pham Truc      10 Truong Chinh, Quan Kien An, Hai Phong   
1   C0000000695    Vu Nguyen          85 Hoang Sa, Huyen Ngoc Hien, Ca Mau   
2   C0000000696     Le Duyen        92 Pham Ngu Lao, Huyen Gia Lam, Ha Noi   
3   C0000000697      Bui Thu     317 Nguyen Trai, Quan Vinh Hai, Nha Trang   
4   C0000000698    Duong Hoa     53 Nguyen Hue, Quan Duong Kinh, Hai Phong   
..          ...          ...                                           ...   
94  C0000000788     Tran Hoa                 17 Le Loi, Quan Phu Vang, Hue   
95  C0000000789  Nguyen Khoa  194 Phan Xich Long, Quan V

Sản phẩm với ID: P01115 đã tồn tại. Bỏ qua.
Sản phẩm với ID: P01116 đã tồn tại. Bỏ qua.
Sản phẩm với ID: P01117 đã tồn tại. Bỏ qua.
Sản phẩm với ID: P01118 đã tồn tại. Bỏ qua.
Sản phẩm với ID: P01119 đã tồn tại. Bỏ qua.
Sản phẩm với ID: P01120 đã tồn tại. Bỏ qua.
Sản phẩm với ID: P01121 đã tồn tại. Bỏ qua.
Sản phẩm với ID: P01122 đã tồn tại. Bỏ qua.
Sản phẩm với ID: P01123 đã tồn tại. Bỏ qua.
Sản phẩm với ID: P01124 đã tồn tại. Bỏ qua.
Sản phẩm với ID: P01125 đã tồn tại. Bỏ qua.
Sản phẩm với ID: P01126 đã tồn tại. Bỏ qua.
Sản phẩm với ID: P01127 đã tồn tại. Bỏ qua.
Sản phẩm với ID: P01128 đã tồn tại. Bỏ qua.
Sản phẩm với ID: P01129 đã tồn tại. Bỏ qua.
Sản phẩm với ID: P01130 đã tồn tại. Bỏ qua.
Sản phẩm với ID: P01131 đã tồn tại. Bỏ qua.
Sản phẩm với ID: P01132 đã tồn tại. Bỏ qua.
Sản phẩm với ID: P01133 đã tồn tại. Bỏ qua.
Sản phẩm với ID: P01134 đã tồn tại. Bỏ qua.
Sản phẩm với ID: P01135 đã tồn tại. Bỏ qua.
Sản phẩm với ID: P01136 đã tồn tại. Bỏ qua.
Sản phẩm với ID: P01137 đã tồn t

    orderdetail_id     order_id sanpham_id  quanlity   price
0     OD0000000827  O0000000564     P01203         5   33.75
1     OD0000000828  O0000000564     P00377         5   17.12
2     OD0000000829  O0000000564     P01958         4   19.99
3     OD0000000830  O0000000564     P00882         4    1.99
4     OD0000000831  O0000000570     P01794         2  189.99
..             ...          ...        ...       ...     ...
403   OD0000001230  O0000000556     P01564         4   12.49
404   OD0000001231  O0000000556     P00937         5   18.95
405   OD0000001232  O0000000556     P01644         4   99.99
406   OD0000001233  O0000000556     P00328         4   17.95
407   OD0000001234  O0000000556     P00529         1  123.99

[408 rows x 5 columns]
Fetched coordinates for Ha Noi: (21.0283334, 105.854041)
Fetched coordinates for Hue: (16.4639321, 107.5863388)
Fetched coordinates for Hai Phong: (20.858864, 106.6749591)
Fetched coordinates for Nha Trang: (12.2431693, 109.1898675)
Fetched coo

'\nscheduler = BackgroundScheduler()\nscheduler.add_job(my_long_running_task, \'interval\', seconds=300)  # Chạy mỗi 300 giây\nscheduler.start()\n\n# Giữ cho ứng dụng chạy\ntry:\n    while True:\n        time.sleep(1)\nexcept (KeyboardInterrupt, SystemExit):\n    scheduler.shutdown()\n    logger.info("Scheduler đã dừng lại!")\n'